In [ ]:
from seleniumbase import Driver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import mysql.connector


In [ ]:
mydb = mysql.connector.connect(
    host = "media-studies-jobs.cux1s0fa60hj.us-east-2.rds.amazonaws.com",
    user = "admin",
    password = "123456789",
    database = "indeed_jobs"
)

cursor = mydb.cursor()
cursor = mydb.cursor(buffered=True)


In [ ]:
driver = Driver(uc = True)

# Setup explicit wait
wait = WebDriverWait(driver, 20)

In [ ]:
jobs_data = []

page = 0  # Initialize page counter
while True:  # Change to an infinite loop
    page_val = page * 12  # Assuming 10 is the pagination step
    url = f"https://www.indeed.com/jobs?q=environmental+science&l=USA&from=searchOnHP&vjk=3242f2585ff64d12{page_val}"
    print("Fetching:", url)

    driver.get(url)

    # Wait for the job listings to be loaded
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        break  # Break the loop if the page doesn't load

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found. Ending search.")
        break  # Break the loop if no jobs are found

    for job in jobs:
        data = {}

        job_link_element = job.find('a', href=True)
        job_link = 'https://www.indeed.com' + job_link_element['href'] if job_link_element else "Not listed"
        data['Apply_Link'] = job_link

        # Visit the job page to extract detailed description
        if job_link != "Not listed":
            driver.get(job_link)
            try:
                link = driver.current_url
                if "?jk=" in link:
                    try:
                        job_id = link.split('?jk=')[1].split('&')[0]
                    except IndexError:
                        job_id = "Error extracting ID"
                else:
                    print('link', link)
                    job_id = "Not listed"

                    try:
                        title = job.find('h2', class_='jobTitle').text.strip()
                    except AttributeError:
                        title = "Not listed"
                    data['Title'] = title

                    try:
                        company = job.find('span', class_="css-92r8pb").text.strip()
                    except AttributeError:
                        company = "Not listed"
                    data['Company'] = company

                    try:
                        location = job.find('div', class_='css-1p0sjhy').text.strip()
                    except AttributeError:
                        location = "Not listed"
                    data['Location'] = location

                    salary = job.find('div', class_='css-1cvo3fd')
                    data['Salary'] = salary.text.strip() if salary else "Not listed"

                    try:
                      wait.until(EC.presence_of_element_located((By.ID, 'jobDescriptionText')))
                      detailed_description = driver.find_element(By.ID, 'jobDescriptionText').text.strip()
                    except (TimeoutException, NoSuchElementException):
                      detailed_description = "Not listed"

                    data['Description'] = detailed_description
                    jobs_data.append(data)

    print(f"Page {page + 1} done.")
    page += 1  # Increment the page counter

# Close the WebDriver after finishing
driver.quit()

# Save data to CSV file
csv_file_path = "jobsindeed.csv"
with open(csv_file_path, "w", newline="", encoding="utf-8") as csv_file:
    fieldnames = ['Title', 'Company', 'Location', 'Salary', 'Description', 'Apply_Link']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for job_data in jobs_data:
        writer.writerow(job_data)

print("Data saved to CSV file successfully.")

Fetching: https://www.indeed.com/jobs?q=media+jobs&l=USA&sc=0kf%3Aexplvl%28ENTRY_LEVEL%29%3B&vjk=1fc92fe6994b5ab5=0
033d97870c791106
dce55d2f62a20b91
Skip job:  d584837266d8bd22
Skip job:  736a0f32a2162d73
d0e7422f1a79b5ec
ca0b90b3fe33e059
Skip job:  a1308eaab615e973
Skip job:  9657f22ef8886cd9
Skip job:  439d442194a11ea3
link https://www.indeed.com/viewjob?cmp=eScribers%2C-LLC&t=Proofreader&jk=753b83d2d75001b8&xpse=SoDK67I3CI8PcwzapZ0LbzkdCdPP&xkcb=SoBx67M3CI7zOfwYl50CbzkdCdPP&vjs=3
Skip job:  Not listed
link https://www.indeed.com/viewjob?cmp=International-College-Counselors&t=Editor&jk=b39d1ec94b4e4770&xpse=SoCr67I3CI8O19SQLJ0LbzkdCdPP&xkcb=SoDs67M3CI7zOfwYl50BbzkdCdPP&vjs=3
Skip job:  Not listed
link https://www.indeed.com/viewjob?cmp=Good-Thomas%2527-Entertainment&t=Social+Media+Manager&jk=9b08bdea179c2096&xpse=SoC467I3CI8OtOwcyB0LbzkdCdPP&xkcb=SoBY67M3CI7zOfwYl50AbzkdCdPP&vjs=3
Skip job:  Not listed
Skip job:  2e410a44bfbef628
Skip job:  0f52533149e31228
link https://www.indeed.c

In [ ]:
jobs_data = []

page = 0  # Initialize page counter
while True:  # Change to an infinite loop
    page_val = page * 12  # Assuming 10 is the pagination step
    url = f"https://www.indeed.com/jobs?q=environmental+science&l=USA&from=searchOnHP&vjk=3242f2585ff64d12{page_val}"
    print("Fetching:", url)

    driver.get(url)

    # Wait for the job listings to be loaded
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        break  # Break the loop if the page doesn't load

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found. Ending search.")
        break  # Break the loop if no jobs are found

    for job in jobs:
        data = {}

        job_link_element = job.find('a', href=True)
        job_link = 'https://www.indeed.com' + job_link_element['href'] if job_link_element else "Not listed"
        data['Apply_Link'] = job_link

        # Visit the job page to extract detailed description
        if job_link != "Not listed":
            driver.get(job_link)
            try:
                link = driver.current_url
                if "?jk=" in link:
                    try:
                        job_id = link.split('?jk=')[1].split('&')[0]
                    except IndexError:
                        job_id = "Error extracting ID"
                else:
                    print('link', link)
                    job_id = "Not listed"

                select_query = "SELECT Job_ID FROM indeed_jobs WHERE Job_ID = %s"
                cursor.execute(select_query, (job_id,))
                result = cursor.fetchone()

                if result is None:
                    data['Job_ID'] = job_id
                    print(job_id)

                    try:
                        title = job.find('h2', class_='jobTitle').text.strip()
                    except AttributeError:
                        title = "Not listed"
                    data['Title'] = title

                    try:
                        company = job.find('span', class_="css-92r8pb").text.strip()
                    except AttributeError:
                        company = "Not listed"
                    data['Company'] = company

                    try:
                        location = job.find('div', class_='css-1p0sjhy').text.strip()
                    except AttributeError:
                        location = "Not listed"
                    data['Location'] = location

                    salary = job.find('div', class_='css-1cvo3fd')
                    data['Salary'] = salary.text.strip() if salary else "Not listed"

                    try:
                      wait.until(EC.presence_of_element_located((By.ID, 'jobDescriptionText')))
                      detailed_description = driver.find_element(By.ID, 'jobDescriptionText').text.strip()
                    except (TimeoutException, NoSuchElementException):
                      detailed_description = "Not listed"

                    data['Description'] = detailed_description
                    jobs_data.append(data)

    print(f"Page {page + 1} done.")
    page += 1  # Increment the page counter

# Close the WebDriver after finishing
driver.quit()

# Save data to CSV file
csv_file_path = "242024.csv"
with open(csv_file_path, "w", newline="", encoding="utf-8") as csv_file:
    fieldnames = ['Title', 'Company', 'Location', 'Salary', 'Description', 'Apply_Link']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for job_data in jobs_data:
        writer.writerow(job_data)

print("Data saved to CSV file successfully.")

In [ ]:

jobs_data = []

page = 0  # Initialize page counter
while page < 5:  # Limit to 5 pages for demonstration purposes
    page_val = page * 12  # Assuming 12 is the pagination step
    url = f"https://www.indeed.com/jobs?q=environmental+science&l=USA&from=searchOnHP&vjk=3242f2585ff64d12{page_val}"
    print("Fetching:", url)

    driver.get(url)

    # Wait for the job listings to be loaded
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.jobsearch-SerpJobCard')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        break  # Break the loop if the page doesn't load

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.find_all('div', class_='jobsearch-SerpJobCard')

    if not jobs:
        print("No more jobs found. Ending search.")
        break  # Break the loop if no jobs are found

    for job in jobs:
        data = {}

        job_link_element = job.find('a', href=True)
        job_link = 'https://www.indeed.com' + job_link_element['href'] if job_link_element else "Not listed"
        data['Apply_Link'] = job_link

        # Visit the job page to extract detailed description
        if job_link != "Not listed":
            driver.get(job_link)
            try:
                job_id = job_link.split('?jk=')[1].split('&')[0]
            except IndexError:
                job_id = "Error extracting ID"

            data['Job_ID'] = job_id

            try:
                title = job.find('h2', class_='title').text.strip()
            except AttributeError:
                title = "Not listed"
            data['Title'] = title

            try:
                company = job.find('span', class_="company").text.strip()
            except AttributeError:
                company = "Not listed"
            data['Company'] = company

            try:
                location = job.find('div', class_='location').text.strip()
            except AttributeError:
                location = "Not listed"
            data['Location'] = location

            salary = job.find('span', class_='salaryText')
            data['Salary'] = salary.text.strip() if salary else "Not listed"

            try:
                detailed_description = driver.find_element(By.ID, 'jobDescriptionText').text.strip()
            except (NoSuchElementException, TimeoutException):
                detailed_description = "Not listed"

            data['Description'] = detailed_description
            jobs_data.append(data)

    print(f"Page {page + 1} done.")
    page += 1  # Increment the page counter

# Close the WebDriver after finishing
driver.quit()

# Save data to CSV file
csv_file_path = "242024.csv"
with open(csv_file_path, "w", newline="", encoding="utf-8") as csv_file:
    fieldnames = ['Title', 'Company', 'Location', 'Salary', 'Description', 'Apply_Link']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for job_data in jobs_data:
        writer.writerow(job_data)

print("Data saved to CSV file successfully.")

# Works fine with no incrimnation

In [ ]:
import csv

# Initialize list to store job data
jobs_data = []

max_pages = 200
page = 0  # Initialize page counter
while page < max_pages:  # Limit the loop to 200 pages
    page_val = page * 12  # Assuming 10 is the pagination step
    url = f"https://www.indeed.com/jobs?q=environmental+science&l=USA&from=searchOnHP&vjk=3242f2585ff64d12{page_val}"
    print("Fetching:", url)

    driver.get(url)

    # Wait for the job listings to be loaded
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        continue  # Skip to the next iteration if the page doesn't load

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found. Ending search.")
        break  # Break the loop if no jobs are found

    for job in jobs:
        data = {}

        job_link_element = job.find('a', href=True)
        job_link = 'https://www.indeed.com' + job_link_element['href'] if job_link_element else "Not listed"
        data['Apply_Link'] = job_link

        # Visit the job page to extract detailed description
        if job_link != "Not listed":
            driver.get(job_link)
            try:
                link = driver.current_url
                if "?jk=" in link:
                    try:
                        job_id = link.split('?jk=')[1].split('&')[0]
                    except IndexError:
                        job_id = "Error extracting ID"
                else:
                    print('link', link)
                    job_id = "Not listed"

                if job_id is None:
                    data['Job_ID'] = job_id
                    print(job_id)

                    try:
                        title = job.find('h2', class_='jobTitle').text.strip()
                    except AttributeError:
                        title = "Not listed"
                    data['Title'] = title

                    try:
                        company = job.find('span', class_="css-92r8pb").text.strip()
                    except AttributeError:
                        company = "Not listed"
                    data['Company'] = company

                    try:
                        location = job.find('div', class_='css-1p0sjhy').text.strip()
                    except AttributeError:
                        location = "Not listed"
                    data['Location'] = location

                    salary = job.find('div', class_='css-1cvo3fd')
                    data['Salary'] = salary.text.strip() if salary else "Not listed"

                    try:
                        detailed_description = job.find('div', class_='summary').text.strip()
                    except AttributeError:
                        detailed_description = "Not listed"
                    data['Description'] = detailed_description

                else:
                    print("Skip job: ", job_id)

            except (TimeoutException, NoSuchElementException):
                detailed_description = "Not listed"
                data['Description'] = detailed_description

            jobs_data.append(data)

    print(f"Page {page + 1} done.")
    page += 1  # Increment the page counter

# Close the WebDriver after finishing
driver.quit()

# Save data to CSV file
csv_file_path = "fuckedup.csv"
with open(csv_file_path, "w", newline="", encoding="utf-8") as csv_file:
    fieldnames = ['Title', 'Company', 'Location', 'Salary', 'Description', 'Apply_Link']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for job_data in jobs_data:
        writer.writerow(job_data)

print("Data saved to CSV file successfully.")


# 1 max page, multiple URLs

In [ ]:
import csv

# Initialize list to store job data
jobs_data = []

max_pages = 1
page = 0  # Initialize page counter
while page < max_pages:
    page_val = page * 12  # Assuming 10 is the pagination step
    url = f"https://www.indeed.com/jobs?q=environmental+science&l=USA&from=searchOnHP&vjk=3242f2585ff64d12{page_val}"
        url = f"https://www.indeed.com/jobs?q=sustainability+jobs&l=USA&sc=0kf%3Aexplvl%28ENTRY_LEVEL%29%3B&vjk=b8a5956f0918959c{page_val}"
    url = f"https://www.indeed.com/jobs?q=sustainable+agriculture&l=USA&vjk=b9b8bf61620566e1{page_val}"
    url = f"https://www.indeed.com/jobs?q=sustainability+consultant&l=USA&vjk=4709ed414e838017{page_val}"

    print("Fetching:", url)

    driver.get(url)

    # Wait for the job listings to be loaded
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        continue  # Skip to the next iteration if the page doesn't load

print(f"Page {page + 1} done.")
page += 1  # Increment the page counter


    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found. Ending search.")
        break  # Break the loop if no jobs are found

    for job in jobs:
        data = {}

        job_link_element = job.find('a', href=True)
        job_link = 'https://www.indeed.com' + job_link_element['href'] if job_link_element else "Not listed"
        data['Apply_Link'] = job_link

        # Visit the job page to extract detailed description
        if job_link != "Not listed":
            driver.get(job_link)
            try:
                link = driver.current_url
                if "?jk=" in link:
                    try:
                        job_id = link.split('?jk=')[1].split('&')[0]
                    except IndexError:
                        job_id = "Error extracting ID"
                else:
                    print('link', link)
                    job_id = "Not listed"

                if job_id is None:
                    data['Job_ID'] = job_id
                    print(job_id)

                    try:
                        title = job.find('h2', class_='jobTitle').text.strip()
                    except AttributeError:
                        title = "Not listed"
                    data['Title'] = title

                    try:
                        company = job.find('span', class_="css-92r8pb").text.strip()
                    except AttributeError:
                        company = "Not listed"
                    data['Company'] = company

                    try:
                        location = job.find('div', class_='css-1p0sjhy').text.strip()
                    except AttributeError:
                        location = "Not listed"
                    data['Location'] = location

                    salary = job.find('div', class_='css-1cvo3fd')
                    data['Salary'] = salary.text.strip() if salary else "Not listed"

                    try:
                        detailed_description = job.find('div', class_='summary').text.strip()
                    except AttributeError:
                        detailed_description = "Not listed"
                    data['Description'] = detailed_description

                else:
                    print("Skip job: ", job_id)

            except (TimeoutException, NoSuchElementException):
                detailed_description = "Not listed"
                data['Description'] = detailed_description

            jobs_data.append(data)


# Close the WebDriver after finishing
driver.quit()

# Save data to CSV file
csv_file_path = "fuckthis.csv"
with open(csv_file_path, "w", newline="", encoding="utf-8") as csv_file:
    fieldnames = ['Title', 'Company', 'Location', 'Salary', 'Description', 'Apply_Link']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for job_data in jobs_data:
        writer.writerow(job_data)

print("Data saved to CSV file successfully.")


In [ ]:


# Setup Chrome options for Selenium
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# Uncomment the next line to run Chrome in headless mode
# options.add_argument('--headless')

# Initialize the Chrome WebDriver
driver = webdriver.Chrome(options=options)

# Setup explicit wait
wait = WebDriverWait(driver, 10)

jobs_data = []
max_pages = 5  # Set a maximum number of pages to scrape
page = 0

while page < max_pages:
    url = f"https://www.indeed.com/jobs?q=sustainability++jobs&l=USA&vjk={page * 10}"
    driver.get(url)

    #https://www.indeed.com/jobs?q=sustainability++jobs&l=USA&vjk=0cdb55029eb47b25

    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found.")
        break

    for job in jobs:
        data = {}
        try:
            data['Title'] = job.find('h2', class_='jobTitle').text.strip()
        except AttributeError:
            data['Title'] = "Not listed"

        try:
            data['Company'] = job.find('span', class_="css-92r8pb").text.strip()
        except AttributeError:
            data['Company'] = "Not listed"

        try:
            data['Location'] = job.find('div', class_='company_location').text.strip()
        except AttributeError:
            data['Location'] = "Not listed"

        skills_section = job.find('div', class_='jobsearch-SkillsWrapper')
        data['Skills'] = ', '.join([skill.text.strip() for skill in skills_section.find_all('span', class_='jobsearch-SkillTag')]) if skills_section else "Not listed"

        try:
            data['Salary'] = job.find('div', class_='css-1cvo3fd').text.strip()
        except AttributeError:
            data['Salary'] = "Not listed"

        try:
            jobDescriptionText = job.find('div', class_='jobsearch-jobDescriptionText')
            data['Description'] = jobDescriptionText.text.strip() if jobDescriptionText else "Not listed"
        except AttributeError:
            data['Description'] = "Not listed"

        try:
            job_link = job.find('a', class_='jcs-JobTitle')['href']
            data['Apply Link'] = 'https://www.indeed.com' + job_link
        except TypeError:
            data['Apply Link'] = "Not listed"

        jobs_data.append(data)

    print(f"Page {page + 1} completed.")
    page += 1

driver.quit()

df = pd.DataFrame(jobs_data)
df.to_csv('indeed_jobs.csv', index=False)
print("Job extraction complete. Data saved to 'indeed_jobs.csv'.")

In [ ]:
jobs_data = []


for i in range(0,2000,10):
    driver.get('https://www.indeed.com/jobs?q=sustainability%20&l=united%20states&start='+str(i))
    driver.implicitly_wait(5)

    #https://www.indeed.com/jobs?q=sustainability++jobs&l=USA&vjk=0cdb55029eb47b25

    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found.")
        break

    for job in jobs:
        data = {}


        try:
          wait.until(EC.presence_of_element_located((By.ID, 'jobDescriptionText')))
          detailed_description = driver.find_element(By.ID, 'jobDescriptionText').text.strip()
        except (TimeoutException, NoSuchElementException):
          detailed_description = "Not listed"

          data['Description'] = detailed_description

        try:
            data['Title'] = job.find('h2', class_='jobTitle').text.strip()
        except AttributeError:
            data['Title'] = "Not listed"

        try:
            data['Company'] = job.find('span', class_="css-92r8pb").text.strip()
        except AttributeError:
            data['Company'] = "Not listed"

        try:
            data['Location'] = job.find('div', class_='company_location').text.strip()
        except AttributeError:
            data['Location'] = "Not listed"

        skills_section = job.find('div', class_='jobsearch-SkillsWrapper')
        data['Skills'] = ', '.join([skill.text.strip() for skill in skills_section.find_all('span', class_='jobsearch-SkillTag')]) if skills_section else "Not listed"

        try:
            data['Salary'] = job.find('div', class_='css-1cvo3fd').text.strip()
        except AttributeError:
            data['Salary'] = "Not listed"


        try:
            wait.until(EC.presence_of_element_located((By.ID, 'jobDescriptionText')))
            detailed_description = driver.find_element(By.ID, 'jobDescriptionText').text.strip()
        except (TimeoutException, NoSuchElementException):
            detailed_description = "Not listed"

        data['Description'] = detailed_description


        try:
            job_link = job.find('a', class_='jcs-JobTitle')['href']
            data['Apply Link'] = 'https://www.indeed.com' + job_link
        except TypeError:
            data['Apply Link'] = "Not listed"

        jobs_data.append(data)
        page += 1
    print(f"Page {page + 1} completed.")


driver.quit()

df = pd.DataFrame(jobs_data)
df.to_csv('clean_scrapped.csv', index=False)
print("Job extraction complete. Data saved to 'indeed_jobs.csv'.")